In [1]:
# BPTI Gen Vel analysis
import MDAnalysis as mda
import numpy as np
from MDAnalysis.analysis import align, rms,pca
import copy

# import pca from scikit-learn

from sklearn.decomposition import PCA
# import tsne
from sklearn.manifold import TSNE

# xMD testing
import pandas as pd
import os
from xMD.xMD import xMD
from xMD.MD_Settings import GROMACS_Settings

settings = GROMACS_Settings()


amber14sb_ff_path = os.path.join(os.getcwd())

# Set the GMXLIB environment variable
os.environ["GMXLIB"] = amber14sb_ff_path



settings = GROMACS_Settings()
settings.suffix = "APO_amber99"
# settings.search = "APO"

print(settings.config)

settings.topology = os.path.join(settings.topology,"BPTI")
print(settings.topology)
# make sure to turn on MPI for HPC 
settings.gmx_mpi_on = False



/home/alexi/miniconda3/envs/xMD/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/alexi/miniconda3/envs/xMD/lib/python3.10/site-packages/Bio/Application/__init__.py:40: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


/home/alexi/Documents/xMD
config
topology/BPTI


In [2]:
def load_cattraj_to_mda(cattraj, top_path:str):
    
    if top_path is None:
        top_path = cattraj.replace(".xtc", ".pdb")

    u = mda.Universe(top_path, cattraj)


    return u


def rmsd_to_df(u: mda.Universe, pdbcode:str, name:str, rep:int):

    df = pd.DataFrame(columns=["pdbcode", "name", "frame", "rmsd", "rep"])

    # Reference for RMSD (first frame)
    ref = u.copy()
    ref.trajectory[0]

    # Calculate RMSD
    rmsd_analysis = rms.RMSD(u, ref, select="name CA")
    rmsd_analysis.run()


    # Populate DataFrame
    for i, frame in enumerate(u.trajectory):
        frame_rmsd = rmsd_analysis.rmsd[i, 2]  # RMSD value
        df_to_add = pd.DataFrame([[pdbcode, name, i, frame_rmsd, rep]], columns=df.columns)
        df = pd.concat([df, df_to_add], ignore_index=True)
    
    return df

In [16]:
def run_multi_analysis(pdbcode: str, names: list, num_reps: int):

    df = pd.DataFrame(columns=["pdbcode", "name", "frame", "rmsd", "rep"])

    paths = []
    lengths = []
    reps = []

    for name in names:
        for rep in range(1,num_reps+1):
            print(f"Running {name} rep {rep}")
            
            md = xMD(settings, name, pdbcode, rep)
            
            data_dir = md.generate_path_structure()
            viz_dir = data_dir.replace("data", "visualisation")

            rep_dir = "R_" + str(rep)


            top_name = "_".join([md.settings.suffix,
                                        md.settings.pdbcode,
                                        str(1)]) + "-nojump" + ".pdb"
            

            cat_traj_name = "_".join([md.settings.suffix,
                                        md.settings.pdbcode]) + "_cat_"+str(rep) + ".xtc"

            cat_traj_path = os.path.join(data_dir, rep_dir, cat_traj_name)
            paths.append(cat_traj_path)
            print(cat_traj_path)
            top_path = os.path.join(data_dir, rep_dir, top_name)

            u = load_cattraj_to_mda(cat_traj_path, top_path)
            
            lengths.append(u.trajectory.n_frames)
            reps.append(rep)

            df_to_add = rmsd_to_df(u, pdbcode, name, rep)

            print(df_to_add)


            df = pd.concat([df, df_to_add], ignore_index=True)




    print(df)
    print(df.tail())

    dim_u = mda.Universe(top_path, *paths)
    ref_frame = dim_u.trajectory[0]
    ref_atoms = dim_u.select_atoms("name CA") 
    #align to first frame
    align.alignto(dim_u, ref_atoms, select="name CA")

    # Get coordinates
    coordinates = np.zeros((dim_u.trajectory.n_frames, dim_u.atoms.n_atoms, 3))
    for i, frame in enumerate(dim_u.trajectory):
        coordinates[i] = frame.positions
    n_frames = coordinates.shape[0]
    n_atoms = coordinates.shape[1]


    # find average coordinates
    average_coordinates = np.mean(coordinates, axis=0)

    # find distance from average
    distance_from_average = np.zeros((n_frames, n_atoms))
    for i in range(n_frames):
        distance_from_average[i] = np.sqrt(np.sum((coordinates[i] - average_coordinates)**2, axis=1))

    

    print(coordinates.shape)
    print(distance_from_average.shape)

    pca = PCA(n_components=2)
    pca_results = pca.fit_transform(distance_from_average)

    # Add the PCA results to the DataFrame
    # Ensure df has the same number of rows as there are frames in the MD trajectory
    df["PCA1"] = pca_results[:, 0]
    df["PCA2"] = pca_results[:, 1]

    tsne = TSNE(n_components=2)

    tsne_results = tsne.fit_transform(distance_from_average)

    df["tSNE1"] = tsne_results[:, 0]
    df["tSNE2"] = tsne_results[:, 1]  

    frames_per_rep = [frame for length in lengths for frame in range(1, length + 1)]
    reps_per_frame = [rep for rep, length in zip(reps, lengths) for _ in range(length)]

    df["frame"] = frames_per_rep
    df["rep"] = reps_per_frame



    return df


In [17]:
test_df = run_multi_analysis("5PTI", ["BPTI_genvel1"],2)

Running BPTI_genvel1 rep 1
Replicate number:  1
Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel1
Trial directory logs:  logs/MD/5PTI/BPTI_genvel1
Trial directory data:  data/MD/5PTI/BPTI_genvel1
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel1
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel1
Environment variables set:  GMXLIB /home/alexi/Documents/xMD
Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel1
Trial directory logs:  logs/MD/5PTI/BPTI_genvel1
Trial directory data:  data/MD/5PTI/BPTI_genvel1
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel1
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel1
data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_cat_1.xtc
    pdbcode          name frame          rmsd rep
0      5PTI  BPTI_genvel1     0  1.770929e-07   1
1      5PTI  BPTI_genvel1     1  3.133262e-01   1
2      5PTI  BPTI_genvel1     2  3.275709e-01   1
3      5PTI  BPTI_genvel1     3  4.289775e-01   1
4      5PTI  BPTI_genv

/home/alexi/miniconda3/envs/xMD/lib/python3.10/site-packages/MDAnalysis/analysis/rms.py:722: DeprecationWarning:

The `rmsd` attribute was deprecated in MDAnalysis 2.0.0 and will be removed in MDAnalysis 3.0.0. Please use `results.rmsd` instead.

/tmp/ipykernel_871411/2819361091.py:29: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/tmp/ipykernel_871411/1348024226.py:44: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/home/alexi/miniconda3/envs/xMD/lib/python3.10/site-packages/MDAnalysis/analysis/

(452, 892, 3)
(452, 892)


In [18]:
print(test_df.to_string())

    pdbcode          name  frame          rmsd  rep        PCA1       PCA2      tSNE1      tSNE2
0      5PTI  BPTI_genvel1      1  1.770929e-07    1  -49.068100  -6.772464 -21.464037   2.254404
1      5PTI  BPTI_genvel1      2  3.133262e-01    1  -43.538061  -4.031072 -20.825464   4.577092
2      5PTI  BPTI_genvel1      3  3.275709e-01    1  -42.629133  -4.814345 -21.626104   4.474173
3      5PTI  BPTI_genvel1      4  4.289775e-01    1  -50.624237  -6.897990 -25.370796   1.611624
4      5PTI  BPTI_genvel1      5  2.695421e-01    1  -49.332834  -6.067375 -23.739828   2.883743
5      5PTI  BPTI_genvel1      6  2.856357e-01    1  -49.769955  -5.049591 -23.970898   1.743317
6      5PTI  BPTI_genvel1      7  2.905204e-01    1  -48.024407  -6.803191 -21.905741   0.843939
7      5PTI  BPTI_genvel1      8  2.522663e-01    1  -48.093776  -8.102637 -22.392462   1.387125
8      5PTI  BPTI_genvel1      9  2.347927e-01    1  -52.237400  -8.500922 -24.498739  -0.696119
9      5PTI  BPTI_genvel1     

In [19]:
# plot rmsd using plotly
import plotly.express as px


In [20]:
#plot split by name  

fig = px.line(test_df, x="frame", y="rmsd", color="rep", facet_col="name", facet_col_wrap=2)
fig.show()

In [9]:
# plot PCA colour by name and rep

fig = px.scatter(test_df, x="PCA1", y="PCA2", color="rep", facet_col="name", facet_col_wrap=2, color_continuous_scale=px.colors.sequential.Viridis)

fig.show()

In [10]:
#plot PCA colour by frame and name

fig = px.scatter(test_df, x="PCA1", y="PCA2", color="frame", facet_col="name", facet_col_wrap=2)

fig.show()



In [12]:
md = xMD(settings, 'BPTI_genvel1', "5PTI", 1)


Replicate number:  1
Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel1
Trial directory logs:  logs/MD/5PTI/BPTI_genvel1
Trial directory data:  data/MD/5PTI/BPTI_genvel1
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel1
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel1
Environment variables set:  GMXLIB /home/alexi/Documents/xMD


In [13]:
md.generate_path_structure()

Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel1
Trial directory logs:  logs/MD/5PTI/BPTI_genvel1
Trial directory data:  data/MD/5PTI/BPTI_genvel1
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel1
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel1


'data/MD/5PTI/BPTI_genvel1'